In [ ]:
#import the Library

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew

from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import roc_auc_score, roc_curve, classification_report

# Exploratory Data

In [ ]:
dataset = pd.read_csv("diabetes.csv")

## Information of Dataset

In [ ]:
dataset.info()

In [ ]:
dataset.head(5)

In [ ]:
dataset.shape

## Checking for missing values

In [ ]:
missing_values = dataset.isnull().sum()
print("Missing Values:")
print(missing_values)

## Diabetical and Non-diabetical Persons

In [ ]:
dataset["Outcome"].value_counts()

#percentage distribution of the "Outcome"
print(100 * dataset["Outcome"].value_counts() / len(dataset))

with_diabetes = dataset['Outcome'].value_counts()[1]
without_diabetes = dataset['Outcome'].value_counts()[0]
print(f"Patients with Diabetes: {with_diabetes}\nPatients without Diabetes: {without_diabetes}")

sns.countplot(x="Outcome", data=dataset)

## Visualizing pregnancies average outcome

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(x='Pregnancies', hue='Outcome', data = dataset)
plt.show()

## Describing data in dataset(mean, std, max)

In [ ]:
describe = dataset.describe().T
describe

## Visualizing the distribution of data in each column

In [ ]:
plt.figure(figsize=(12, 6))
for i, col in enumerate(dataset.columns[:-1]):
    plt.subplot(2, 4, i + 1)
    sns.histplot(dataset[dataset['Outcome'] == 1][col], kde=True, label='Diabetes', color='blue')
    sns.histplot(dataset[dataset['Outcome'] == 0][col], kde=True, label='No Diabetes', color='orange')
    plt.title(f"Distribution of {col}")
    plt.legend()
plt.tight_layout()
plt.show()

 Dengan melihat plot ini dapat membandingkan sebaran data antara pasien dengan diabetes dan tanpa diabetes untuk setiap fitur. Misalnya, jika melihat "Distribution of Glucose," bagaimana sebaran kadar glukosa berbeda antara dua kelompok pasien.

## Target VS features

In [ ]:
dataset.groupby('Outcome').mean()

In [ ]:
plt.figure(figsize=(16,8))

for i, col in enumerate(dataset.columns[:-1]):
    plt.subplot(2, 4, i + 1)
    sns.barplot(x='Outcome', y=dataset[col], data=dataset)
    plt.title(f"{col} vs. Outcome")

plt.tight_layout()
plt.show()

## Correlation of dataset

In [ ]:
dataset.corr()

## Visualizing the correlation of the dataset with Heat Map.

In [ ]:
f, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(dataset.corr(), annot=True, linewidths=0.5, linecolor="black", fmt=".2f", ax=ax)
plt.show()

## Visualizing the correlation of the dataset with Pair Plot.

In [ ]:
sns.pairplot(dataset, hue="Outcome")
plt.show()

## Visualizing the correlation of the dataset with ClusterMap.

In [ ]:
corr_matrix = dataset.corr()
sns.clustermap(corr_matrix, annot = True, fmt = ".2f")
plt.title("Correlation Between Features")
plt.show()

# Data Pre-Processing

Pemrosesan data (data preprocessing) adalah langkah-langkah yang dilakukan untuk membersihkan, mengatur, dan mempersiapkan data mentah sebelum digunakan dalam analisis atau pemodelan. Ini bertujuan untuk memastikan data siap digunakan dalam algoritma pembelajaran mesin agar data tersebut memberikan hasil yang lebih akurat.

## Outliers Analysis

Outliers analysis adalah proses identifikasi dan penanganan nilai-nilai ekstrem atau tidak biasa dalam dataset. Outliers adalah data yang secara signifikan berbeda dari sebagian besar data lain dalam dataset. Analisis outliers penting dalam pemrosesan data dan analisis statistik karena outliers dapat mempengaruhi hasil analisis dengan signifikan.

In [ ]:
'''
 Kode dibawah digunakan untuk membuat box plot (di dalam loop) untuk setiap
 kolom (fitur) dalam dataset. Box plot adalah jenis visualisasi yang berguna
 untuk menunjukkan distribusi data, serta mengidentifikasi outliers jika ada
'''

for c in dataset.columns:
    plt.figure()
    sns.boxplot(x = c, data = dataset, orient = "v")

## Create a function to clear outlier data in the dataset.

In [ ]:
describe = dataset.describe()

def cleanColumn(data,columns, thr = 2):
    column_desc = describe[columns]

    q3 = column_desc[6]
    q1 = column_desc[4]
    IQR = q3 - q1

    top_limit_clm = q3 + thr * IQR
    bottom_limit_clm = q1 - thr * IQR

    filter_clm_bottom = bottom_limit_clm < data[columns]
    filter_clm_top = data[columns] < top_limit_clm

    filters = filter_clm_bottom & filter_clm_top

    data = data[filters]

    print("{} of dataset after column {}".format(data.shape, columns))

    return data

Melakukan pembersihan data dengan menghapus data yang dianggap sebagai outliers (nilai yang jauh dari rata-rata atau yang dianggap sebagai data anomali). Anda melakukan ini untuk setiap kolom dalam dataset.

In [ ]:
for i in dataset.columns:
    data = cleanColumn(dataset, i)

In [ ]:
print("New data shape: ", dataset.shape)

## Feature Engineering

Proses menciptakan variabel (fitur) tambahan dari data yang ada atau mengubah fitur yang sudah ada dalam rangka meningkatkan kualitas model pembelajaran mesin.

In [ ]:
for i in dataset.columns:
    plt.figure()
    sns.distplot(dataset[i], fit = norm)
    plt.show()

1. Histogram: Grafik histogram diatas membagi rentang data ke dalam beberapa interval atau "bin" dan menghitung berapa banyak data yang jatuh ke dalam setiap bin. Grafik ini memberikan pandangan tentang sebaran nilai-nilai data.

2. Kurva Distribusi Normal (Fit=norm): Jika melihat garis kurva distribusi normal yang cocok dengan histogram, ini adalah upaya untuk memeriksa sejauh mana data
sesuai dengan distribusi normal (kurva Gaussian). Jika garis ini mendekati bentuk histogram, maka data cenderung memiliki distribusi normal.

In [ ]:
# Plot ini digunakan untuk memeriksa sejauh mana distribusi data cocok dengan
# distribusi normal. Setiap plot Q-Q membandingkan distribusi data aktual
# terhadap distribusi normal yang diharapkan.

for i in dataset.columns:
    plt.figure()
    stats.probplot(dataset[i], plot = plt)
    plt.title(i)
    plt.show()

1. Probability Plot (Q-Q Plot): Ini adalah plot yang digunakan untuk membandingkan distribusi data aktual dengan distribusi normal yang diharapkan. Jika semua titik dalam plot ini berada di garis diagonal, itu menunjukkan bahwa data Anda mengikuti distribusi normal.

2. Garis Diagonal: Garis diagonal pada plot adalah representasi distribusi normal yang diharapkan. Jika titik-titik data cocok dengan garis ini, maka data Anda adalah normal.

3. Titik-titik Data: Titik-titik dalam plot adalah representasi distribusi aktual dari data Anda. Jika titik-titik ini mengikuti garis diagonal, itu menunjukkan bahwa data Anda adalah normal. Jika titik-titik ini terpencar dari garis diagonal, itu menunjukkan bahwa data memiliki distribusi yang berbeda dari distribusi normal.

In [ ]:
# Mengidentifikasi fitur-fitur dalam dataset yang memiliki sebaran data yang
# condong atau "skewed." Ketika data condong, ini dapat memengaruhi kinerja
# model machine learning, terutama model yang mengasumsikan bahwa data
# terdistribusi normal.

skewed_feats = dataset.apply(lambda x: skew(x.dropna())).sort_values(ascending = False)
skewness = pd.DataFrame(skewed_feats, columns = ["Skewed"])
skewness

Jika skewness positif, maka ini menunjukkan bahwa distribusi data pada fitur tersebut cenderung miring ke kanan (positif), yang berarti terdapat banyak data yang tersebar di nilai-nilai yang lebih tinggi. Skewness negatif akan menunjukkan bahwa distribusi data cenderung miring ke kiri (negatif), dengan data-data yang lebih banyak di nilai-nilai yang lebih rendah. Skewness mendekati nol menunjukkan bahwa distribusi data kurang miring dan lebih simetris.

##Splitting the Dataset into the Training set and Test Set

In [ ]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [ ]:
# Dalam kasus ini, sebanyak 25% dari data akan digunakan sebagai data uji,
# sementara sisanya (75%) akan digunakan sebagai data latih.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [ ]:
print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)
print("y_train shape: ", y_train.shape)
print("y_test shape: ", y_test.shape)

## Apply StandardScaler and RobustScaler on the X_train and X_test.


**Standard Scaller**
1. Fungsi: Mengubah data sehingga memiliki rata-rata nol (0) dan deviasi standar satu (1).
2. Kapan digunakan: Cocok untuk data yang memiliki distribusi normal dan tidak memiliki pencilan (outliers) yang signifikan.
3. Cara kerja: Mengurangkan rata-rata semua nilai dalam data dari setiap nilai, dan kemudian membaginya dengan deviasi standar. Ini menghasilkan data yang terdistribusi dengan rata-rata di sekitar nol dan sebaran seragam.

**Robust Scaller**
1. Fungsi: Mengubah data sehingga tahan terhadap pencilan (outliers) dengan menjaga distribusi data tetap seragam.
2. Kapan digunakan: Cocok untuk data yang memiliki pencilan (outliers) yang signifikan atau tidak terdistribusi normal.
3. Cara kerja: Menggunakan median dan deviasi kuartil (IQR) untuk mengubah data. IQR adalah perbedaan antara kuartil atas dan kuartil bawah, yang membuat metode ini lebih tahan terhadap data ekstrem.

In [ ]:
scaler = StandardScaler()
SSX_train = scaler.fit_transform(X_train)
SSX_test = scaler.transform(X_test)

Rscaler =  RobustScaler()
RSX_train = Rscaler.fit_transform(X_train)
RSX_test = Rscaler.transform(X_test)

train_test = [SSX_train, RSX_train, SSX_test, RSX_test]

# Machine Learning Classification Models

In [ ]:
# Import Logistic Regression Models
from sklearn.linear_model import LogisticRegression

# Import K-Nearest Neighbor Models
from sklearn.neighbors import KNeighborsClassifier

# Import Support Vector Machine Models
from sklearn.svm import SVC

# Import Naive Bayes Models
from sklearn.naive_bayes import GaussianNB

# Import Decision Tree Classification Models
from sklearn.tree import DecisionTreeClassifier

# Import Random Forest Classification Models
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, confusion_matrix

## Create a dataframe for the ML models.

In [ ]:
result_ml_data = pd.DataFrame(columns = ["Model_Name", "SS_Score", "RS_Score"])

In [ ]:
model_name = ["LR", "KNN", "SVM", "NB", "DT", "RF"] # Abridgment Names of Models
result_ml_data["Model_Name"] = model_name

In [ ]:
result_ml_data

#Logistic Regression

Logistic Regression adalah cara untuk memprediksi sesuatu yang hanya memiliki dua kemungkinan hasil. Misalnya, apakah pasien memiliki penyakit tertentu atau tidak. Algoritma ini menggunakan matematika untuk menemukan hubungan antara faktor-faktor yang Anda miliki (seperti usia, berat badan, tekanan darah, dll.) dan probabilitas terjadinya hasil yang Anda minati (misalnya, probabilitas seseorang memiliki penyakit). Ini menghasilkan output yang berada dalam rentang 0 hingga 1 dan dapat diinterpretasikan sebagai probabilitas.

**Proses tuning hyperparameter menggunakan GridSearchCV pada model logistic regression**
Tujuan utamanya adalah untuk meningkatkan akurasi, mencegah overfitting atau underfitting, dan menyesuaikan model dengan kebutuhan tugas tertentu. Dengan tuning, mencari konfigurasi yang membuat model ML bekerja paling baik pada data uji yang belum pernah dilihat sebelumnya. Ini membantu memastikan bahwa model dapat menggeneralisasi dengan baik dan efisien dalam menyelesaikan tugasnya.

In [ ]:

for i in range(0,2):

    # Membuat object model logistic regression
    logreg = LogisticRegression(random_state = 0)

    # Untuk menjaga dataset latih dan uji asli tidak berubah selama proses
    # tuning hyperparameter maka dibuat data latih dan uji khusus model LR
    # yang berdasarkan data diatas
    X_train_logreg, X_test_logreg, y_train_logreg, y_test_logreg = X_train, X_test, y_train, y_test

    # proses melatih model
    logreg.fit(X_train_logreg, y_train_logreg)

    '''
    Dengan menggabungkan berbagai nilai parameter ini, menciptakan daftar
    kombinasi hyperparameter yang berbeda yang akan diuji selama pencarian
    parameter. GridSearchCV akan mencoba setiap kombinasi dan mengukur kinerja
    model menggunakan metrik yang ditentukan (dalam hal ini, "accuracy")
    untuk menemukan parameter yang memberikan hasil terbaik.
    '''

    logreg_params_grid = [{"penalty" : ["l1","l2"], "solver" : ["newton-cg", "lbfgs", "liblinear", "sag", "saga"],
             "multi_class" : ["auto","ovr","multinomial"]}]

    grid_search_logreg = GridSearchCV(estimator = logreg, param_grid = logreg_params_grid, scoring = "accuracy", cv = 4)
    grid_search_logreg_result = grid_search_logreg.fit(train_test[i], y_train_logreg)
    y_pred_logreg = grid_search_logreg_result.predict(train_test[i+2])

    best_param = grid_search_logreg_result.best_params_
    best_score = grid_search_logreg_result.best_score_

    print("Best prameter of grid search function: ", best_param)
    print("Best score of grid search function: ", best_score)

    if (i == 0):
        result_ml_data["SS_Score"][0] = accuracy_score(y_pred_logreg, y_test_logreg)
    else:
        result_ml_data["RS_Score"][0] = accuracy_score(y_pred_logreg, y_test_logreg)

Hasil ini menunjukkan bahwa setelah melakukan pencarian parameter dengan GridSearchCV, dua set parameter yang berbeda memberikan hasil yang serupa dalam hal akurasi. Ini dapat terjadi karena beberapa kombinasi parameter dapat menghasilkan hasil yang mirip dalam hal kinerja model.

## Machine Learning Models Assestment

In [ ]:
result_ml_data

## Predicting the Test results

In [ ]:
logreg.predict(X_test_logreg)
# print(np.concatenate((y_pred_logreg.reshape(len(y_pred_logreg), 1), y_test.reshape(len(y_test_logreg),1)),1))

## Visualizing Actual Values and Prediction Values with Confussion Matrix

In [ ]:
def plot_confussion_matrix_logreg(y_test_logreg, y_pred_logreg):
  acc = round(accuracy_score(y_test_logreg, y_pred_logreg), 2)
  cm = confusion_matrix(y_pred=y_pred_logreg, y_true=y_test_logreg)
  sns.heatmap(cm, annot=True, fmt=".0f", xticklabels=["Positive", "Negative"], yticklabels=["Positive", "Negative"])
  plt.xlabel("Prediction")
  plt.ylabel("Actual")
  plt.title("Accuracy Score: {0}".format(acc), size=10)
  plt.show()

plot_confussion_matrix_logreg(y_test_logreg, y_pred_logreg)
print(classification_report(y_test_logreg, y_pred_logreg))


1. True Positives (TP): Ini mengindikasikan bahwa model logistic regression dengan benar memprediksi 118 pasien sebagai positif (mengidap penyakit diabetes).

2. False Positives (FP): Ini mengindikasikan bahwa model logistic regression salah memprediksi 12 pasien sebagai positif (mengidap penyakit diabetes), padahal sebenarnya mereka negatif.

3. False Negatives (FN): Ini mengindikasikan bahwa model logistic regression salah memprediksi 26 pasien sebagai negatif (tidak mengidap penyakit diabetes), padahal sebenarnya mereka positif.  

4. True Negatives (TN): Ini mengindikasikan bahwa model logistic regression dengan benar memprediksi 36 pasien sebagai negatif (tidak mengidap penyakit diabetes).

Akurasi (Accuracy): Akurasi mengukur sejauh mana model yg benar secara keseluruhan. Ini dihitung sebagai (TP + TN) / (TP + TN + FP + FN).

Presisi (Precision): Presisi mengukur sejauh mana prediksi positif model adalah benar. Ini dihitung sebagai TP / (TP + FP).

Recall (Sensitivitas): Recall mengukur sejauh mana model Anda dapat mengidentifikasi semua kasus positif yang sebenarnya. Ini dihitung sebagai TP / (TP + FN).

F1-Score (F1-Score): F1-Score adalah rata-rata harmonik dari presisi dan recall. Ini memberikan keseimbangan antara kedua metrik tersebut.

## ROC Curve Visualization

ROC adalah grafik yang digunakan untuk menilai kinerja model klasifikasi, seperti yang digunakan dalam machine learning. Ini mengukur kemampuan model untuk membedakan antara kelas positif dan negatif. Grafik ini menampilkan perbandingan antara tingkat True Positive (ketepatan memprediksi kelas positif) dan tingkat False Positive (kesalahan dalam memprediksi kelas positif) pada berbagai ambang batas atau threshold yang berbeda. Semakin dekat kurva ROC ke sudut kiri atas, semakin baik kemampuan model untuk membedakan antara kedua kelas.

In [ ]:
# Menghitung probabilitas prediksi
y_pred_prob = logreg.predict_proba(X_test_logreg)[:, 1]

# Menghitung ROC curve
fpr, tpr, thresholds = roc_curve(y_test_logreg, y_pred_prob)

# Menghitung area di bawah kurva ROC (AUC)
roc_auc = roc_auc_score(y_test_logreg, y_pred_prob)

# Membuat plot untuk ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()

Secara kualitatif dapat menginterpretasikan AUC sebagai berikut:

1. AUC = 0.5: Model tidak memiliki kemampuan pemisahan antara kelas positif dan negatif; itu sebanding dengan pemilihan kelas secara acak.
2. 0.5 < AUC < 1: Model memiliki kemampuan memisahkan kelas dengan berbagai tingkat. Semakin tinggi AUC, semakin baik modelnya.
3. AUC = 1: Model sempurna dan mampu memisahkan kelas positif dan negatif tanpa kesalahan.

Dalam konteks diatas AUC = 0.86 menunjukkan bahwa model logistic regression memiliki kemampuan yang baik dalam memisahkan kelas positif dan negatif.

# K-Nearest Neighbors

K-Nearest Neighbors adalah algoritma yang memutuskan hasil prediksi berdasarkan mayoritas kelas tetangga terdekat. Jadi, jika Anda ingin memprediksi sesuatu (misalnya, kategori sebuah bunga), algoritma ini akan melihat tetangga terdekat dari bunga yang akan diprediksi dan memilih kategori mayoritas dari tetangga-tetangga tersebut. Ini adalah pendekatan yang sederhana dan intuitif dalam machine learning.

In [ ]:
# Proses tuning hyperparameter menggunakan GridSearchCV pada model KNN

for i in range(0,2):

    # Membuat objek model k-nearest neighbors

    knn = KNeighborsClassifier(n_neighbors = 5)
    X_train_knn, X_test_knn, y_train_knn, y_test_knn = X_train, X_test, y_train, y_test

    knn.fit(X_train_knn, y_train_knn)

    knn_params_grid = {"n_neighbors" : range(1,100), "weights" : ["uniform", "distance"], "algorithm" : ["auto", "ball_tree", "kd_tree", "brute"], "p" : [1,2]}

    grid_knn = GridSearchCV(estimator = knn, param_grid = knn_params_grid, scoring = "accuracy", cv = 4)
    grid_knn_search = grid_knn.fit(train_test[i], y_train_knn)
    y_pred_knn = grid_knn.predict(train_test[i+2])

    best_param_grid_knn = grid_knn_search.best_params_
    best_score_grid_knn = grid_knn_search.best_score_

    print("Best prameter of gridsearch function: ", best_param_grid_knn)
    print("Best score of gridsearch function: ",best_score_grid_knn)

    if (i == 0):
        result_ml_data["SS_Score"][1] = accuracy_score(y_pred_knn, y_test_knn)
    else:
        result_ml_data["RS_Score"][1] = accuracy_score(y_pred_knn, y_test_knn)

## Machine Learning Models Assestment

In [ ]:
result_ml_data

## Predicting the Test results

In [ ]:
knn.predict(X_test_knn)

## Visualizing Actual Values and Prediction Values with Confussion Matrix

In [ ]:
def plot_confussion_matrix_knn(y_test_knn, y_pred_knn):
  acc = round(accuracy_score(y_test_knn, y_pred_knn), 2)
  cm = confusion_matrix(y_pred=y_pred_knn, y_true=y_test_knn)
  sns.heatmap(cm, annot=True, fmt=".0f", xticklabels=["Positive", "Negative"], yticklabels=["Positive", "Negative"])
  plt.xlabel("Prediction")
  plt.ylabel("Actual")
  plt.title("Accuracy Score: {0}".format(acc), size=10)
  plt.show()

plot_confussion_matrix_knn(y_test_knn, y_pred_knn)
print(classification_report(y_test_knn, y_pred_knn))

1. True Positives (TP): Ini mengindikasikan bahwa model KNN dengan benar memprediksi 118 pasien sebagai positif (mengidap penyakit diabetes).

2. False Positives (FP): Ini mengindikasikan bahwa model KNN salah memprediksi 12 pasien sebagai positif (mengidap penyakit diabetes), padahal sebenarnya mereka negatif.

3. False Negatives (FN): Ini mengindikasikan bahwa model KNN salah memprediksi 31 pasien sebagai negatif (tidak mengidap penyakit diabetes), padahal sebenarnya mereka positif.  

4. True Negatives (TN): Ini mengindikasikan bahwa model KNN dengan benar memprediksi 31 pasien sebagai negatif (tidak mengidap penyakit diabetes).

## ROC Curve Visualization

In [ ]:
# Menghitung probabilitas prediksi
y_pred_prob = knn.predict_proba(X_test_knn)[:, 1]

# Menghitung ROC curve
fpr, tpr, thresholds = roc_curve(y_test_knn, y_pred_prob)

# Menghitung area di bawah kurva ROC (AUC)
roc_auc = roc_auc_score(y_test_knn, y_pred_prob)

# Membuat plot untuk ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()

# Support Vector Machine

SVM adalah algoritma yang mencari garis batas terbaik untuk memisahkan dua kelas data. Ia mencoba untuk membuat margin (jarak) terbesar antara garis batas dan titik-titik data terdekat. Algoritma ini juga dapat digunakan dalam masalah regresi untuk memprediksi angka (bukan hanya kategori). SVM dikenal sebagai salah satu algoritma klasifikasi yang kuat dan serbaguna dalam machine learning.

In [ ]:
# Proses tuning hyperparameter menggunakan GridSearchCV pada model SVM

for i in range(0,2):

    ## Membuat objek model Support Vector Machine

    svm = SVC(kernel="linear", random_state=0, probability=True)
    X_train_svm, X_test_svm, y_train_svm, y_test_svm = X_train, X_test, y_train, y_test

    svm.fit(X_train_svm, y_train_svm)

    svm_params_grid = [{'C':[1,2,3,4,5],'kernel':['linear']},
        {'C':[1,2,3,4,5],'kernel':['rbf'],'gamma':[1,0.5,0.1,0.01,0.001]},
        {'C':[1,2,3,4,5],'kernel':['poly'],'degree':[1,2,3,4,5,6,7],'gamma':[1,0.5,0.1,0.01,0.001]}]

    grid_svm = GridSearchCV(estimator = svm, param_grid = svm_params_grid, scoring = "accuracy", cv = 4)
    grid_search_svm = grid_svm.fit(train_test[i], y_train_svm)
    y_pred_svm = grid_search_svm.predict(train_test[i+2])


    best_param_grid = grid_search_svm.best_params_
    best_score_grid = grid_search_svm.best_score_

    print("Best prameter of gridsearch function: ", grid_search_svm.best_params_)
    print("Best score of gridsearch function: ",grid_search_svm.best_score_ )

    if (i == 0):
        result_ml_data["SS_Score"][2] = accuracy_score(y_pred_svm,y_test_svm)
    else:
        result_ml_data["RS_Score"][2] = accuracy_score(y_pred_svm,y_test_svm)

## Machine Learning Models Assestment

In [ ]:
result_ml_data

## Predicting the Test results

In [ ]:
svm.predict(X_test_svm)

## Visualizing Actual Values and Prediction Values with Confussion Matrix

In [ ]:
def plot_confussion_matrix_svm(y_test_svm, y_pred_svm):
  acc = round(accuracy_score(y_test_svm, y_pred_svm), 2)
  cm = confusion_matrix(y_pred=y_pred_svm, y_true=y_test_svm)
  sns.heatmap(cm, annot=True, fmt=".0f", xticklabels=["Positive", "Negative"], yticklabels=["Positive", "Negative"])
  plt.xlabel("Prediction")
  plt.ylabel("Actual")
  plt.title("Accuracy Score: {0}".format(acc), size=10)
  plt.show()

plot_confussion_matrix_svm(y_test_svm, y_pred_svm)
print(classification_report(y_test_svm, y_pred_svm))

1. True Positives (TP): Ini mengindikasikan bahwa model SVM dengan benar memprediksi 118 pasien sebagai positif (mengidap penyakit diabetes).

2. False Positives (FP): Ini mengindikasikan bahwa model SVM salah memprediksi 12 pasien sebagai positif (mengidap penyakit diabetes), padahal sebenarnya mereka negatif.

3. False Negatives (FN): Ini mengindikasikan bahwa model SVM salah memprediksi 27 pasien sebagai negatif (tidak mengidap penyakit diabetes), padahal sebenarnya mereka positif.  

4. True Negatives (TN): Ini mengindikasikan bahwa model SVM dengan benar memprediksi 35 pasien sebagai negatif (tidak mengidap penyakit diabetes).

## ROC Curve Visualization

In [ ]:
# Menghitung probabilitas prediksi
y_pred_prob = svm.predict_proba(X_test_svm)[:, 1]

# Menghitung ROC curve
fpr, tpr, thresholds = roc_curve(y_test_svm, y_pred_prob)

# Menghitung area di bawah kurva ROC (AUC)
roc_auc = roc_auc_score(y_test_svm, y_pred_prob)

# Membuat plot untuk ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()

# Gaussian Naive Bayes

GNB digunakan untuk mengklasifikasikan data dengan menghitung probabilitas kemungkinan kelas berdasarkan nilai fitur yang diamati. Ini adalah algoritma klasifikasi yang efisien.

In [ ]:
for i in range(0,2):
    # Membuat object model naive bayes
    gaussian_nb = GaussianNB()

    X_train_nb, X_test_nb, y_train_nb, y_test_nb = X_train, X_test, y_train, y_test

    gaussian_nb.fit(train_test[i], y_train_nb)
    y_pred_nb = gaussian_nb.predict(train_test[i+2])

    bnb_cvs = cross_val_score(estimator = gaussian_nb, X = train_test[i], y = y_train_nb, cv = 5)

    print("GaussianNB Accuracy: ", accuracy_score(y_pred_nb, y_test_nb))

    print("GaussianNB Test Score: ", gaussian_nb.score(train_test[i+2], y_test_nb))
    print("GaussianNB Train Score: ", gaussian_nb.score(train_test[i], y_train_nb))

    print("GaussianNB Cross Validation Mean: ", bnb_cvs.mean())
    print("GaussianNB Cross Validation Std: ", bnb_cvs.std())
    print("-------------------------------------------")

    if (i == 0):
        result_ml_data["SS_Score"][3] = accuracy_score(y_pred_nb, y_test_nb)
    else:
        result_ml_data["RS_Score"][3] = accuracy_score(y_pred_nb, y_test_nb)

## Machine Learning Models Assestment

In [ ]:
result_ml_data

## Predicting the Test Result

In [ ]:
gaussian_nb.predict(X_test_nb)

## Visualizing Actual Values and Prediction Values with Confussion Matrix

In [ ]:
def plot_confussion_matrix_gnb(y_test_nb, y_pred_nb):
  acc = round(accuracy_score(y_test_nb, y_pred_nb), 2)
  cm = confusion_matrix(y_pred=y_pred_nb, y_true=y_test_nb)
  sns.heatmap(cm, annot=True, fmt=".0f", xticklabels=["Positive", "Negative"], yticklabels=["Positive", "Negative"])
  plt.xlabel("Prediction")
  plt.ylabel("Actual")
  plt.title("Accuracy Score: {0}".format(acc), size=10)
  plt.show()

plot_confussion_matrix_gnb(y_test_nb, y_pred_nb)
print(classification_report(y_test_nb, y_pred_nb))

1. True Positives (TP): Ini mengindikasikan bahwa model naive bayes dengan benar memprediksi 114 pasien sebagai positif (mengidap penyakit diabetes).

2. False Positives (FP): Ini mengindikasikan bahwa model naive bayes salah memprediksi 16 pasien sebagai positif (mengidap penyakit diabetes), padahal sebenarnya mereka negatif.

3. False Negatives (FN): Ini mengindikasikan bahwa model naive bayes salah memprediksi 29 pasien sebagai negatif (tidak mengidap penyakit diabetes), padahal sebenarnya mereka positif.  

4. True Negatives (TN): Ini mengindikasikan bahwa model naive bayes dengan benar memprediksi 33 pasien sebagai negatif (tidak mengidap penyakit diabetes).

## ROC Curve Visualization

In [ ]:
# Menghitung probabilitas prediksi
y_pred_prob = gaussian_nb.predict_proba(X_test_nb)[:, 1]

# Menghitung ROC curve
fpr, tpr, thresholds = roc_curve(y_test_nb, y_pred_prob)

# Menghitung area di bawah kurva ROC (AUC)
roc_auc = roc_auc_score(y_test_nb, y_pred_prob)

# Membuat plot untuk ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()

#Decision Tree Classification

Pohon Keputusan adalah algoritma yang membuat keputusan berdasarkan serangkaian pertanyaan. Ini mirip dengan bermain tebakan di mana setiap pertanyaan membantu kita mencapai keputusan akhir. Contohnya, dalam masalah klasifikasi, pohon keputusan dapat bertanya, "Apakah pasien memiliki demam?" Jika jawabannya "Ya," pohon akan mengarahkan ke pertanyaan berikutnya, dan seterusnya, hingga mencapai prediksi akhir seperti "Pasien memiliki flu." Pohon ini membantu kita memahami bagaimana keputusan dibuat dari data.

In [ ]:
for i in range(0,2):
    # Membuat object model decision tree
    dtc = DecisionTreeClassifier()
    X_train_dtc, X_test_dtc, y_train_dtc, y_test_dtc = X_train, X_test, y_train, y_test

    dtc.fit(X_train_dtc, y_train_dtc)

    dtc_params = {"criterion" : ["gini", "entropy","log_loss"], "splitter" : ["best", "random"], "max_features" : ["auto", "sqrt", "log2"]}

    grid_dtc = GridSearchCV(estimator = dtc, param_grid = dtc_params, scoring = "accuracy", cv = 4)
    grid_dtc_search = grid_dtc.fit(train_test[i], y_train_dtc)
    y_pred_dtc = grid_dtc.predict(train_test[i+2])

    best_param_grid_dtc = grid_dtc_search.best_params_
    best_score_grid_dtc = grid_dtc_search.best_score_

    print("Best prameter of gridsearch function: ", best_param_grid_dtc)
    print("Best score of gridsearch function: ",best_score_grid_dtc)

    if (i == 0):
        result_ml_data["SS_Score"][4] = accuracy_score(y_pred_dtc, y_test_dtc)
    else:
        result_ml_data["RS_Score"][4] = accuracy_score(y_pred_dtc, y_test_dtc)

## Machine Learning Models Assestment

In [ ]:
result_ml_data

## Predicting the Test Result

In [ ]:
dtc.predict(X_test_dtc)

## Visualizing Actual Values and Prediction Values with Confussion Matrix

In [ ]:
def plot_confussion_matrix_dtc(y_test_dtc, y_pred_dtc):
  acc = round(accuracy_score(y_test_dtc, y_pred_dtc), 2)
  cm = confusion_matrix(y_test_dtc, y_pred_dtc)
  sns.heatmap(cm, annot=True, fmt=".0f", xticklabels=["Positive", "Negative"], yticklabels=["Positive", "Negative"])
  plt.xlabel("Prediction")
  plt.ylabel("Actual")
  plt.title("Accuracy Score: {0}".format(acc), size=10)
  plt.show()

plot_confussion_matrix_dtc(y_test_dtc, y_pred_dtc)
print(classification_report(y_test_dtc, y_pred_dtc))

1. True Positives (TP): Ini mengindikasikan bahwa model DT dengan benar memprediksi 97 pasien sebagai positif (mengidap penyakit diabetes).

2. False Positives (FP): Ini mengindikasikan bahwa model DT salah memprediksi 33 pasien sebagai positif (mengidap penyakit diabetes), padahal sebenarnya mereka negatif.

3. False Negatives (FN): Ini mengindikasikan bahwa model DT salah memprediksi 20 pasien sebagai negatif (tidak mengidap penyakit diabetes), padahal sebenarnya mereka positif.  

4. True Negatives (TN): Ini mengindikasikan bahwa model DT dengan benar memprediksi 42 pasien sebagai negatif (tidak mengidap penyakit diabetes).

## ROC Curve Visualization

In [ ]:
# Menghitung probabilitas prediksi
y_pred_prob = dtc.predict_proba(X_test_dtc)[:, 1]

# Menghitung ROC curve
fpr, tpr, thresholds = roc_curve(y_test_dtc, y_pred_prob)

# Menghitung area di bawah kurva ROC (AUC)
roc_auc = roc_auc_score(y_test_dtc, y_pred_prob)

# Membuat plot untuk ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()

# Random Forest Classification

Random Forest adalah algoritma machine learning yang memadukan banyak pohon keputusan untuk menghasilkan prediksi yang lebih akurat. Ini seperti mengumpulkan banyak pandangan berbeda untuk membuat keputusan yang lebih baik. Daripada mengandalkan satu pohon keputusan, Random Forest menggunakan "hutan" berisi banyak pohon yang secara kolektif memberikan hasil yang lebih andal. Ini membantu mengatasi overfitting dan meningkatkan kinerja model.

In [ ]:
for i in range(0,2):

    # Membuat object model random forest classification

    rfc = RandomForestClassifier()
    X_train_rfc, X_test_rfc, y_train_rfc, y_test_rfc = X_train, X_test, y_train, y_test

    rfc.fit(X_train_rfc, y_train_rfc)

    param_rfc = {"n_estimators" : range(1,50), "criterion" : ["gini", "entropy","log_loss"], "max_features" : ["sqrt","log2", None],
             "class_weight" : ["balanced", "balanced_subsample"]}

    grid_rfc = GridSearchCV(estimator = rfc, param_grid = param_rfc, scoring = "accuracy", cv = 4)
    grid_rfc_search = grid_rfc.fit(train_test[i], y_train_rfc)
    y_pred_rfc = grid_rfc.predict(train_test[i+2])

    best_parm_grid_rfc = grid_rfc_search.best_params_
    best_score_grid_rfc = grid_rfc_search.best_score_

    print("Best prameter of gridsearch function: ", best_parm_grid_rfc)
    print("Best score of gridsearch function: ",best_score_grid_rfc)

    if (i == 0):
        result_ml_data["SS_Score"][5] = accuracy_score(y_pred_rfc, y_test_rfc)
    else:
        result_ml_data["RS_Score"][5] = accuracy_score(y_pred_rfc, y_test_rfc)

## Machine Learning Models Assestment

In [ ]:
result_ml_data

## Predicting the Test Result

In [ ]:
rfc.predict(X_test_rfc)

## Visualizing Actual Values and Prediction Values with Confussion Matrix

In [ ]:
def plot_confussion_matrix_rfc(y_test_rfc, y_pred_rfc):
  acc = round(accuracy_score(y_test_rfc, y_pred_rfc), 2)
  cm = confusion_matrix(y_pred=y_pred_rfc, y_true=y_test_rfc)
  sns.heatmap(cm, annot=True, fmt=".0f", xticklabels=["Positive", "Negative"], yticklabels=["Positive", "Negative"])
  plt.xlabel("Prediction")
  plt.ylabel("Actual")
  plt.title("Accuracy Score: {0}".format(acc), size=10)
  plt.show()

plot_confussion_matrix_dtc(y_test_rfc, y_pred_rfc)
print(classification_report(y_test_rfc, y_pred_rfc))

1. True Positives (TP): Ini mengindikasikan bahwa model RF dengan benar memprediksi 115 pasien sebagai positif (mengidap penyakit diabetes).

2. False Positives (FP): Ini mengindikasikan bahwa model RF salah memprediksi 15 pasien sebagai positif (mengidap penyakit diabetes), padahal sebenarnya mereka negatif.

3. False Negatives (FN): Ini mengindikasikan bahwa model RF salah memprediksi 23 pasien sebagai negatif (tidak mengidap penyakit diabetes), padahal sebenarnya mereka positif.  

4. True Negatives (TN): Ini mengindikasikan bahwa model RF dengan benar memprediksi 39 pasien sebagai negatif (tidak mengidap penyakit diabetes).

## ROC Curve Visualization

In [ ]:
# Menghitung probabilitas prediksi
y_pred_prob = rfc.predict_proba(X_test_rfc)[:, 1]

# Menghitung ROC curve
fpr, tpr, thresholds = roc_curve(y_test_rfc, y_pred_prob)

# Menghitung area di bawah kurva ROC (AUC)
roc_auc = roc_auc_score(y_test_rfc, y_pred_prob)

# Membuat plot untuk ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()

## Visualizing Result with Standart Scaler

In [ ]:
plt.figure(figsize=(10,7))
sns.barplot(x=result_ml_data["Model_Name"], y=result_ml_data["SS_Score"])
plt.xticks(rotation=0)
plt.xlabel("Model_Name")
plt.ylabel("SS_Score")
plt.title("Result with Standart Scaler")

## Visualizing Result with Robust Scaller

In [ ]:
plt.figure(figsize=(10,7))
sns.barplot(x=result_ml_data["Model_Name"], y=result_ml_data["RS_Score"])
plt.xticks(rotation=0)
plt.xlabel("Model_Name")
plt.ylabel("RS_Score")
plt.title("Result with Robust Scaler")